In [1]:
import gc
import os
import pickle
import random
import joblib
import pandas as pd
# import polars as pd
from tqdm import tqdm

import numpy as np
from sklearn.metrics import average_precision_score as APS
from sklearn.model_selection import StratifiedKFold

import torch
from torch.utils.data import TensorDataset, Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from pytorch_lightning.callbacks.lr_monitor import LearningRateMonitor

from pytorch_lightning import LightningModule
from pytorch_lightning import LightningDataModule, Trainer
# seed_everything
from pytorch_lightning.callbacks import (
    ModelCheckpoint, 
    EarlyStopping,
    ModelCheckpoint,
    RichModelSummary,
    RichProgressBar,
)
from pytorch_lightning.loggers import TensorBoardLogger
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

from funcs.utils import find_latest_ckpt_path, del_old_ckpt_path
from funcs.calc_descriptor import calc_rdkit_descriptors, calc_ecfp4_descriptors
from funcs.tokenize import tokenize_smiles

import warnings
warnings.simplefilter('ignore')

In [2]:
import os
from pathlib import Path

BASE_DIR = Path(os.getcwd()) / './../'
DATA_DIR = BASE_DIR / "data"
OUTPUT_DIR = DATA_DIR / f"rdkit_descriptors"
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

SHRUNKEN_DATA_DIR = DATA_DIR / "shrunken-train-set"

TRAIN_PATH = SHRUNKEN_DATA_DIR / "train.parquet"

In [3]:
df_train = pd.read_parquet(TRAIN_PATH, columns=["molecule_smiles"])

In [4]:
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors

descriptor_names = [desc_name for desc_name, _ in Descriptors.descList]
descriptor_calculation = MoleculeDescriptors.MolecularDescriptorCalculator(descriptor_names)

smiles_list = df_train["molecule_smiles"].tolist()[:1000]
mols_list = [Chem.MolFromSmiles(smiles) for smiles in smiles_list]

temp = []
for mol in tqdm(mols_list):
    temp.append(descriptor_calculation.CalcDescriptors(mol))

100%|██████████| 1000/1000 [00:06<00:00, 163.60it/s]


In [7]:
len(df_train) / 160 / 60 / 60

170.86043402777779

In [8]:
    
df_test = pd.read_parquet(DATA_DIR / 'test.parquet')
df_test

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name
0,295246830,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C=Cc1ccc(N)cc1,C=Cc1ccc(N)cc1,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ccc(C=C...,BRD4
1,295246831,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C=Cc1ccc(N)cc1,C=Cc1ccc(N)cc1,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ccc(C=C...,HSA
2,295246832,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C=Cc1ccc(N)cc1,C=Cc1ccc(N)cc1,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ccc(C=C...,sEH
3,295246833,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C=Cc1ccc(N)cc1,CC(O)Cn1cnc2c(N)ncnc21,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ncnc3c2...,BRD4
4,295246834,C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C=Cc1ccc(N)cc1,CC(O)Cn1cnc2c(N)ncnc21,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ncnc3c2...,HSA
...,...,...,...,...,...,...
1674891,296921721,[N-]=[N+]=NCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc...,Nc1noc2ccc(F)cc12,COC1CCC(CCN)CC1,COC1CCC(CCNc2nc(Nc3noc4ccc(F)cc34)nc(N[C@@H](C...,HSA
1674892,296921722,[N-]=[N+]=NCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc...,Nc1noc2ccc(F)cc12,COC1CCC(CCN)CC1,COC1CCC(CCNc2nc(Nc3noc4ccc(F)cc34)nc(N[C@@H](C...,sEH
1674893,296921723,[N-]=[N+]=NCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc...,Nc1noc2ccc(F)cc12,NCc1cccs1,[N-]=[N+]=NCCC[C@H](Nc1nc(NCc2cccs2)nc(Nc2noc3...,BRD4
1674894,296921724,[N-]=[N+]=NCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc...,Nc1noc2ccc(F)cc12,NCc1cccs1,[N-]=[N+]=NCCC[C@H](Nc1nc(NCc2cccs2)nc(Nc2noc3...,HSA


In [3]:
# 各分子の記述子を計算する


In [ ]:
def calc_rdkfp_descriptors(bb_dicts:dict):

    

    return df_RDkfp


def calc_ecfp4_descriptors(bb_dicts:dict, radius:int=2, nBits:int=1024):

    idx_list = bb_dicts.keys()
    smiles_list = [bb_dicts[idx] for idx in idx_list]
    mols_list = [Chem.MolFromSmiles(smiles) for smiles in smiles_list]

    ECFP4 = [AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits) for mol in mols_list]
    df_ECFP4 = pd.DataFrame(np.array(ECFP4, int),index=idx_list)
    
    return df_ECFP4